In [1]:
import numpy as np

### Create triagle matrix

In [50]:
class TriMatrix:
    def __init__(self):
        self.matrix = np.array([])

    def _poli_sum_on_row(self, base_number, row_index):
        sum = 0
        for index, value in enumerate(self.matrix[row_index]):
            sum += value*base_number**index
        return sum

    def call_triagle(self, level, below_num):
        process_num = below_num - 1
        self.matrix = np.zeros((level, level+1))

        current_row_index = 0
        current_col_index = 1
        while True:
            if (current_row_index == 0) and (current_col_index == 1):
                self.matrix[current_row_index][current_col_index] = 1
                    
            elif current_col_index == 0:
                self.matrix[current_row_index][current_col_index] = - self._poli_sum_on_row(process_num, current_row_index)

            elif current_col_index == 1:
                self.matrix[current_row_index][current_col_index] = 1 - np.sum(self.matrix[current_row_index])

            else:
                self.matrix[current_row_index][current_col_index] = current_row_index/current_col_index * self.matrix[current_row_index-1][current_col_index-1]

            if (current_row_index == level-1) and (current_col_index == 0):
                break
            
            current_row_index += 1
            current_col_index += 1

            if current_row_index == level:
                current_row_index = level + 1 - current_col_index
                current_col_index = 0

        return self.matrix



tri = TriMatrix()
tri.call_triagle(3, 2)
        

array([[-1.        ,  1.        ,  0.        ,  0.        ],
       [-1.        ,  0.5       ,  0.5       ,  0.        ],
       [-1.        ,  0.16666667,  0.5       ,  0.33333333]])

### Example: Find the sum formula:

$$f(x) = \sum^{x}_{x=3}{\sum^{x}_{x=2}{x+1}}$$

In [52]:
input_matrix = np.array([[1, 1]])

input_matrix @ tri.call_triagle(input_matrix.shape[1], 2) @ tri.call_triagle(input_matrix.shape[1]+1, 3)

array([[-3.        , -1.16666667,  1.        ,  0.16666667]])

$$\Rightarrow f(x) = -3 - \frac{7}{6}x + x^2 + \frac{1}{6}x^3$$